### Environment Setting

In [1]:
import numpy as np
import torch
import torch.nn.functional as F
import math
import matplotlib.pyplot as plt
import cv2
import os, time, pickle

def np_psnr(original, upscaled):
    max_pixel = np.max(original)
    mse = np.square(original - upscaled).mean()
    return 20 * np.log10(max_pixel) - 10 * np.log10(mse)

def torch_psnr(original, upscaled):
    max_pixel = torch.max(original)
    mse = torch.square(original - upscaled).mean()
    return 20 * torch.log10(max_pixel) - 10 * torch.log10(mse)

# Each xii -> hard-coded convolution kernel
x00 = np.array(
      [[0, 0, 0, 0], 
       [0, 1, 0, 0], 
       [0, 0, 0, 0], 
       [0, 0, 0, 0]])
x10 = np.array(
      [[0, -9, 0, 0], 
       [0, 111, 0, 0], 
       [0, 29, 0, 0], 
       [0, -3, 0, 0]])/128
x20 = np.array(
      [[0, -1, 0, 0], 
       [0, 9, 0, 0], 
       [0, 9, 0, 0], 
       [0, -1, 0, 0]])/16
x30 = np.flip(x10, 0)

x01 = np.transpose(x10)
# x11 = np.array(
#       [[81, -999, -261, 27], 
#        [-999, 12321, 3219, -333], 
#        [-261, 3219, 841, -87], 
#        [27, -333, -87, 9]])/2**14

# downscaled 1/32: 
x11 = np.array(
       [[3, -31,  -8,   0],
       [-31, 385, 101, -10],
       [ -8, 101,  26,  -3],
       [  0, -10,  -3,   0]])/2**9

# x21 = np.array(
#       [[9, -111, -29, 3], 
#        [-81, 999, 261, -27], 
#        [-81, 999, 261, -27], 
#        [9, -111, -29, 3]])/2**11

# downscaled 1/4:
x21 = np.array([[  2, -28,  -7,   1],
       [-20, 250,  65,  -7],
       [-20, 250,  65,  -7],
       [  2, -28,  -7,   1]])/2**9


x31 = np.flip(x11, 0)

x02 = np.transpose(x20)
x12 = np.transpose(x21)
x22 = np.array(
      [[1, -9, -9, 1], 
       [-9, 81, 81, -9], 
       [-9, 81, 81, -9], 
       [1, -9, -9, 1]])/256
x32 = np.flip(x12, 0)

x03 = np.transpose(x30)
x13 = np.transpose(x31)
x23 = np.transpose(x32)
x33 = np.flip(x13, 0)

transform = torch.tensor(np.array([
    [x00, x01, x02, x03], 
    [x10, x11, x12, x13], 
    [x20, x21, x22, x23], 
    [x30, x31, x32, x33]
    ]))

def get_bicubic_spline(input):
    # Apply bicubic spline to input
    # Input size: (Channel, H, W)

    return torch.einsum('cij,hwij->chw', input, transform)


kernel1 = np.array(
      [[0, -9, 0, 0], 
       [0, 111, 0, 0], 
       [0, 29, 0, 0], 
       [0, -3, 0, 0]])
kernel2 = np.array(
      [[0, -1, 0, 0], 
       [0, 9, 0, 0], 
       [0, 9, 0, 0], 
       [0, -1, 0, 0]])
kernel3 = np.array([[  3, -31,  -8,   0],
       [-31, 385, 101, -10],
       [ -8, 101,  26,  -3],
       [  0, -10,  -3,   0]])
kernel4= np.array([[  2, -28,  -7,   1],
       [-20, 250,  65,  -7],
       [-20, 250,  65,  -7],
       [  2, -28,  -7,   1]])
kernel5= np.array(
      [[1, -9, -9, 1], 
       [-9, 81, 81, -9], 
       [-9, 81, 81, -9], 
       [1, -9, -9, 1]])

### Automated TestBench Generation for each Kernel
iverilog -g2012 -o kernel_tb/kernel_1.out sim/kernel_1_tb.sv hdl/bicubic_interpolation.sv

In [2]:
#Parameter Settings
N_test = 50
clk_cycle = 10

basic_header = lambda name: f"""
$dumpfile("test/{name}.vcd"); //file to store value change dump (vcd)
$dumpvars(0,{name}_tb); //store everything at the current level and below
$display("Starting Sim"); //print nice message
clk_in = 0; //initialize clk (super important)
rst_in = 0; //initialize rst (super important)

#10;  //wait a little bit of time at beginning
rst_in = 1; //reset system
#10; //hold high for a few clock cycles
rst_in=0;

"""
basic_tail = """
$display("Finishing Sim"); //print nice message
$finish;
"""

def pretty_print_patch(array, indent=""):
    input_line = []
    for v in range(4):
        row = []
        for h in range(4):
            row.append(str(int(array[v][h])))
        input_line.append("[" + ", ".join(row) + "]")

    return  indent+"[" + (",\\n"+indent).join(input_line) + "]"

def pretty_print_result():
    output=""
    for row in range(4):
        output += f'$display("\\t[%d, %d, %d, %d]\\t[%d, %d, %d, %d]\\t[%d, %d, %d, %d]", '
        format_list = []
        for channel in ['r_out', 'g_out', 'b_out']:
            for col in range(4):
                format_list.append(f'{channel}[{row}][{col}]')
        output += ", ".join(format_list)
        output += ');\n'
    return output

In [4]:
# Parameter Input
param_dict = {
    'kernel_1': [4, 128,  kernel1],
    'kernel_2': [4, 16,   kernel2],
    'kernel_3': [4, 2**9, kernel3],
    'kernel_4': [4, 2**9, kernel4],
    'kernel_5': [4, 256,  kernel5],
}
N_test = 100
clk_cycle = 10

for name in param_dict.keys():

    # Create testbench (initial) block
    out_string = basic_header(name)

    # Create Test cases
    pipeline, scale, kernel = param_dict[name]
    shift = int(np.log2(scale))-2
    min_val, max_val = 0, 2**8-1

    test_case = [np.zeros((4,4)), np.full((4,4), 32), np.full((4,4), 63),
                np.array([[63, 0, 0, 63],
                        [0, 63, 63, 0],
                        [0, 63, 63, 0], 
                        [63, 0, 0, 63]]), # Worst case (maximum value)
                np.array([[0, 63, 63, 0],
                        [63, 0, 0, 63],
                        [63, 0, 0, 63], 
                        [0, 63, 63, 0]]), # Worst case (minimum value)
                ]
    for i in range(N_test):
        test_case.append(np.random.randint(0, 63, (4,4)))
    test_case += [None, ]* pipeline

    # Format Testbench
    for i, curr_case in enumerate(test_case):
        
        if i >= pipeline:
            prev_case = test_case[i-pipeline]
            matmul = int(np.sum(kernel * prev_case))
            output = matmul >> shift

            # if negative   (two most significant bits should be 'b11), 
            # elif over 255 (two most significant bits should be 'b10)
            if output < min_val:
                output = "val>"+str(2**8+2**7-1)
            elif output > max_val:
                output = f"{str(2**8+2**7)}>val>255"

            print_kernel = pretty_print_patch(prev_case)
                

            out_string+=f'$display("Input: \\n{print_kernel}");\n'
            out_string+=f'$display("Expect: {output}, Result: %d", pixel_out);\n'
            out_string+=f'$display("");\n'
            

        if curr_case is not None:
            # set pixel
            for v in range(4):
                for h in range(4):
                    out_string += f"pixel_array_in[{v}][{h}] = {int(curr_case[v][h])};\n"
        out_string += f"#{clk_cycle};\n\n"

    out_string += basic_tail

    indented_string = out_string.replace('\n','\n\t\t')

    with open(f"test/{name}_tb_rawstring.txt", "w") as f:
        f.write(indented_string)


### Automatic Testcase Generation for Bicubic Interpolation Module

In [3]:
name = 'bicubic_interpolation'

# Create testbench (initial) block
out_string = basic_header(name)

# Create Test cases
pipeline = 5
min_val, max_val = 0, 2**8-1

full_31 = np.full((1,4,4), 31)
full_63 = np.full((1,4,4), 63)
max_63 = np.array([[[63, 0, 0, 63],
                    [0, 63, 63, 0],
                    [0, 63, 63, 0], 
                    [63, 0, 0, 63]]]).astype(int)
min_63 = np.array([[[0, 63, 63, 0],
                    [63, 0, 0, 63],
                    [63, 0, 0, 63], 
                    [0, 63, 63, 0]]]).astype(int)

test_case = [np.zeros((3,4,4)), 
            np.concatenate((full_31, full_63, full_31), axis=0),
            np.concatenate((full_31,)*3, axis=0),
            np.concatenate((max_63//2, max_63, max_63//2), axis=0), # Worst case (maximum value)
            np.concatenate((min_63//2, min_63, min_63//2), axis=0), # Worst case (minimum value)
            ]

for i in range(N_test):
    test_case.append(np.concatenate(
        [np.random.randint(0, 31, (1,4,4)),
        np.random.randint(0, 63, (1,4,4)),
        np.random.randint(0, 31, (1,4,4))],
        axis = 0)
    )
test_case += [None, ]* pipeline

# Format Testbench
for i, curr_case in enumerate(test_case):
    
    if i >= pipeline:
        prev_case = test_case[i-pipeline]
        normalized_patch = np.array([
            prev_case[0]*8,
            prev_case[1]*4,
            prev_case[2]*8
        ])
        convolved = np.einsum('cij,hwij->chw', normalized_patch, transform)
        # clip to [0, 255] range
        expected  = np.clip(convolved, 0, 255)

        # pretty format for printing
        expected_print_list = np.array([pretty_print_patch(array, indent="\\t").split('\\n') for array in expected], dtype=str)
        transposed = expected_print_list.transpose()
        expected_print = "\\n".join(["".join(row) for row in transposed])

        out_string+= f'$display("Case {i+1}:");\n'
        out_string+= f'$display("\\tExpect: \\n{expected_print}, \\n\\tResult:");\n'
        out_string+= pretty_print_result()
        out_string+= f'$display("");\n'
        

    if curr_case is not None:
        # set pixel
        for v in range(4):
            for h in range(4):
                red, green, blue = curr_case[:, v, h].astype(int)
                # pixel = {5'b red, 6'b green, 5'b blue}
                current_pixel    = blue + (green << 5)+ (red << 11)
                out_string += f"pixel_array_in[{v}][{h}] = {current_pixel};\n"
    out_string += f"#{clk_cycle};\n\n"

out_string += basic_tail

indented_string = out_string.replace('\n','\n\t\t')

with open(f"test/{name}_tb_rawstring.txt", "w") as f:
    f.write(indented_string)


### Use Image Sample

In [ ]:

# Manual kernel for downscaling area x4 times (using area interpolation)
# Following kernel is used to overcome the issue of misalignment problem using
#      the cv2's innate downscale method
downscale_kernel = torch.Tensor([[1/4, 1/2, 1/2, 1/2, 1/4], 
                   [1/2, 1, 1, 1, 1/2], 
                   [1/2, 1, 1, 1, 1/2], 
                   [1/2, 1, 1, 1, 1/2],
                   [1/4, 1/2, 1/2, 1/2, 1/4]]).reshape((1,1,5,5)).to(torch.double)/16

def bicubic_spline_image(target):
    # (H, W, C) -> (C, H, W) for Torch convention
    target = target.transpose((2,0,1))
    C, H, W = target.shape

    # Crop to cleanly manageable size
    H, W = ((H-2)//16)*16+2, ((W-2)//16)*16+2
    target = target[:,:H,:W]

    target = torch.Tensor(target).to(torch.double)

    # proper 4x downscaling (area interpolation)
    downscaled = F.conv2d(target.reshape((C, 1, H, W)), 
                          downscale_kernel, 
                          stride=4, padding=0).squeeze()
    
    C, dH, dW = downscaled.shape
    output = torch.zeros((C, (dH-3)*4, (dW-3)*4))

    # sequential upscaling (equivalent to the FPGA implementation)
    start= time.time()
    for h in range(dH-3):
        for w in range(dW-3):
            patch = downscaled[:,h:h+4, w:w+4]
            bicubic_result = get_bicubic_spline(patch)
            output[:, 4*h:4*(h+1),4*w:4*(w+1)] = bicubic_result
    duration = time.time()-start
    
    # align target to the upscaled coordinate (margins cropped)
    target_aligned = target[:, 6:H-8, 6:W-8]

    original, upscaled = [
        image.to(torch.float32).numpy().transpose(1,2,0)
            for image in [target_aligned, output]
    ]
    psnr =  torch_psnr(target_aligned, output).item()

    return psnr, original, upscaled, duration
        

W_target = 320 # resize width target
nan = [float('inf'), float('-inf')]

path = './imagenet/'
file_list = os.listdir(path)

N = len(file_list)
n_query = 10

keys = [ 'spline', 'conv','bilinear', 'nearest']
path_list = []
hist = {
    key: [] for key in keys
}
query_time = {
    key: 0. for key in keys
}
method_dict = {
    'conv': cv2.INTER_CUBIC, 'bilinear': cv2.INTER_LINEAR, 
    'nearest': cv2.INTER_NEAREST
}

sample_result = {key: 0 for key in keys+['target',]}

step = N//n_query
for n in range(N-11, N, step):
    if n % 50 == 0 :
        print(f'{n//step}/{N//step}')
    f_path = os.path.join(path, file_list[n])

    # Load Image
    img = cv2.imread(f_path)
    H, W, C = img.shape

    # Set resolution to OV7670 camera (320 x 240) -- preserve image proportion
    H_target = int((H * W_target/W)) # resize to (%, 320)
    resized_img = cv2.resize(img, (W_target, H_target), interpolation=cv2.INTER_AREA)

    spline_psnr, target, upscaled, duration = bicubic_spline_image(resized_img)

    hist['spline'].append(spline_psnr)
    path_list.append(f_path)
    query_time['spline'] += duration
    sample_result['spline'] = upscaled
    sample_result['target'] = target

    H, W, C = target.shape
    if H%4 != 0 or W%4 != 0:
        raise ValueError(f"Resized image should be multiple of 4, but got {H, W}")
    
    downscaled_img = cv2.resize(target, (W//4, H//4), interpolation=cv2.INTER_AREA)

    # Compare with other traditional methods
    for method, val in method_dict.items():
        start= time.time()
        upscaled = cv2.resize(downscaled_img, 
                                (W, H),
                                interpolation = val)
        duration = time.time() - start
        psnr = np_psnr(target, upscaled)

        hist[method] = hist[method] + [psnr,]
        query_time[method] += duration
        sample_result[method] = upscaled
        
avg_psnr = {
    method: np.array(hist[method]).mean() for method in keys
}
query_time = {
    method : query_time[method]/ n_query for method in keys
}

# with open('demo_result.pickle', 'wb') as handle:
#     pickle.dump({'path': path_list,
#                  'hist': hist,
#                  'time': query_time,
#                  'sample': sample_result}, handle, protocol=pickle.HIGHEST_PROTOCOL)

print(avg_psnr)
print(query_time)

fig, ax =plt.subplots(1,5) # ax array
sample = {
    key: sample_result[key].astype(int) for key in sample_result.keys()
}
title = ['GT','Bicubic Spline', 'Bicubic Conv.',
            'Bilinear', 'Nearest Neighbor']

for i, method in enumerate(['target', 'spline', 'conv', 'bilinear', 'nearest']):
    ax[i].imshow(sample[method][:,:,::-1])

    ax[i].set_title(title[i])
    ax[i].set_xticks([])
    ax[i].set_yticks([])
    if i > 0:
        psnr = hist[method][-1]
        ax[i].set_xlabel(f'({psnr:.1f} dB)')

plt.show()